# Übung 3

## 1) Gesichtserkennung mit Naive Bayes

In [15]:
import pandas as pd
import numpy as np
import matplotlib as mp
import tarfile
import os.path
import urllib.request
import skimage
import scipy as sp
from random import shuffle

In [5]:
filename = "lfw-funneled.tgz"
url = "http://vis-www.cs.umass.edu/lfw/" + filename
if not os.path.isfile("../02/" + filename):
    urllib.request.urlretrieve(url, filename)
    dateDownloaded = !date
    print(dateDownloaded)

In [6]:
tar = tarfile.open("../02/" + filename)
tar.extractall()
tar.close()
foldername = "lfw_funneled"

In [17]:
persons = {}
for personName in os.listdir(foldername):
    current_person = foldername + "/" + personName
    if os.path.isdir(current_person):
        if len(os.listdir(current_person)) >= 70:
            shuffled_dir_list = os.listdir(current_person)
            shuffle(shuffled_dir_list)
            persons.update({personName: [current_person +"/"+item for item in shuffled_dir_list] })
persons.keys()

dict_keys(['Ariel_Sharon', 'Colin_Powell', 'Gerhard_Schroeder', 'George_W_Bush', 'Tony_Blair', 'Hugo_Chavez', 'Donald_Rumsfeld'])

In [24]:
from skimage import io
from skimage import transform

resized_images = []
resized_names = []
resized_last_images = []
resized_last_names = []

for person_name, person in zip(persons.keys(), persons.values()):
    path = person.copy()
    size = (int)(len(path) * 0.6)
    for picture_path in path:
        picture = io.imread(picture_path,  as_grey=True)
        picture = picture[55:195, 75:175]
        resized_image = transform.resize(picture, ((int)(picture.shape[0]/8), (int)(picture.shape[1]/8))).flatten()
        if picture_path not in path[size:len(path)]:
            resized_images.append(resized_image)
            resized_names.append(person_name)
        else:
            resized_last_images.append(resized_image)
            resized_last_names.append(person_name)

resized_images = np.array(resized_images)
resized_names = np.array(resized_names)
resized_last_images = np.array(resized_last_images)
resized_last_names = np.array(resized_last_names)
        
resized_images.shape, resized_names.shape, resized_last_images.shape, resized_last_names.shape

((770, 204), (770,), (518, 204), (518,))

Führen Sie anschließend eine Hauptkomponentenanalyse auf den Trainingsdaten durch und projizieren Sie sowohl Trainings- als auch Testbilder auf die ersten 7 Eigengesichter.

In [30]:
training_data_mean = resized_images.mean(axis=0)
centered_training_data = resized_images.copy()
centered_training_data=centered_training_data-training_data_mean

U, D, Vt = np.linalg.svd(resized_images)

eigenvalues = np.square(D)

centered_test_data = resized_last_images.copy()
centered_test_data=centered_test_data-training_data_mean

test_images = np.dot(Vt[:7], centered_test_data.T).T
training_images = np.dot(Vt[:7], centered_training_data.T).T

test_images.shape, training_images.shape

((518, 7), (770, 7))

Trainieren Sie Ihren GNB-Klassifikator auf dem Trainingsdatensatz als “George-W.-Bush-Detektor”, d.h. alle zu dieser Person gehörigen Bilder werden mit 1 gelabelt, alle sonstigen mit –1

In [37]:
test_names = resized_last_names
training_names = resized_names
training_labels = []
for training_image, training_name in zip(training_images, training_names):
    if training_name == "George_W_Bush":
        training_labels.append(1)
    else:
         training_labels.append(-1) 
training_labels.count(1), training_labels.count(-1)

(318, 452)

In [49]:
training_not_bush = np.empty((0,training_images.shape[1]))
training_bush =  np.empty((0,training_images.shape[1]))
for training_name, training_label, training_image in zip(training_names, training_labels, training_images):
    if training_label == 1:
        training_bush = np.vstack((training_bush, training_image))
    else:
        training_not_bush = np.vstack((training_not_bush, training_image))  

Folie 18: Mittelwert und Standardabweichung

In [50]:
training_not_bush_mean = np.mean(training_not_bush, axis=0)
training_bush_mean =np.mean(training_bush, axis=0)
test_images_mean = np.mean(test_images, axis=0)

training_not_bush_std = np.std(training_not_bush, axis=0)
training_bush_std =np.std(training_bush, axis=0)
test_images_std = np.std(test_images, axis=0)

training_not_bush_mean, training_bush_mean, test_images_mean, training_not_bush_std, training_bush_std, test_images_std

(array([ 0.01794558, -0.02942273, -0.04640468, -0.03352693, -0.01647695,
         0.09350531,  0.0753956 ]),
 array([-0.02550755,  0.04182099,  0.06595885,  0.04765463,  0.02342007,
        -0.13290692, -0.10716608]),
 array([ 0.0154012 , -0.03415545,  0.05320721, -0.01264308, -0.03497061,
        -0.00717286, -0.02515022]),
 array([ 0.94465711,  0.77693594,  0.75431473,  0.60104409,  0.48523402,
         0.42158808,  0.38092372]),
 array([ 0.92905173,  0.8740246 ,  0.5647243 ,  0.58528228,  0.49104588,
         0.32651634,  0.33208179]),
 array([ 0.93430386,  0.78501948,  0.67469988,  0.57245414,  0.46640445,
         0.39242559,  0.3464955 ]))

A-Priori Wahrscheinlichkeiten

In [52]:
apriori_bush = training_bush.shape[0] / (training_bush.shape[0]+training_not_bush.shape[0])
apriori_not_bush = training_not_bush.shape[0] / (training_bush.shape[0]+training_not_bush.shape[0])
apriori_bush, apriori_not_bush

(0.412987012987013, 0.587012987012987)

Werten Sie Ihren Klassifikator sowohl auf den Trainings- wie auf den unabhängigen Testdaten aus

Bestimmen Sie dafür jeweils die Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate

## Konfiguration

In [28]:
%load_ext version_information
%version_information numpy, pandas, matplotlib, skimage

Software versions
Python 3.5.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.1.0
OS Linux 4.2.0 27 generic x86_64 with debian jessie sid
numpy 1.11.1
pandas 0.18.1
matplotlib 1.5.3
skimage 0.12.3
Sun Nov 27 13:58:46 2016 CET